In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import warnings

warnings.filterwarnings('ignore')

from multiprocessing import get_context
import tensorflow as tf
import time
from decimal import Decimal, ROUND_HALF_UP
from scipy.optimize import brentq

from DNNFM_functions import *
from comp_m_functions import static_factor_obs, cov_e_poet

def round(x):
    return int(Decimal(x).to_integral_value(rounding=ROUND_HALF_UP))

#---------------- Main ----------------

def DNN_FM_main(data, data_factor, architecture=1, const_err_cov=2.5, use_CV_err=False, eval_type='frob'):

    data_dm = data
    data_factor_dm = data_factor
    
    # Obtain optimal tuning parameter based on cross-validation or pre-specified values
    opt = opt_hyper_parameters(data=data, data_F=data_factor, architecture=architecture, const_err_cov=const_err_cov, 
                               use_CV_err=use_CV_err, eval_type=eval_type)
    # Compute DNN-FM based on optimal hyper-parameters
    res_DNN_FM = DNN_FM_core(data=data_dm, data_factor=data_factor_dm, architecture=architecture, opt=opt)

    # c_err_min = DNN_FM_cov_e_cmin(data=data_dm, data_factor=data_factor_dm, DNN_model=res_DNN_FM['neural_net'])

    print(opt['const_err_cov'])
    res_DNN_FM_cov = DNN_FM_cov(data=data_dm, data_factor=data_factor_dm, DNN_model=res_DNN_FM['neural_net'], 
                                c_err_cov=opt['const_err_cov'], check_eig=False)
    
    res_DNN_FM.update(res_DNN_FM_cov)

    return res_DNN_FM


#---------------- Functions ----------------

# Core function for creating Neural Network

def DNN_FM_core(data, data_factor, architecture, opt):

    num_n, num_s = data.shape
    num_f = data_factor.shape[1]

    # Create neural network specifications

    if architecture == 1:
        inter_layer = False

        n_layers = 1
        d_rate = 0.2

        if inter_layer:
            c_temp = 2
        else:
            c_temp = 1

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+c_temp)
        activation_functions = [np.nan] * (n_layers+c_temp)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            hidden_layer_s[mm] = 512
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        if inter_layer:
            dropout_rates_tr[len(dropout_rates_tr)-1] = d_rate
            activation_functions[len(activation_functions)-2] = 'relu'
            activation_functions[len(activation_functions)-1] = 'relu'
        else:
            activation_functions[len(activation_functions)-1] = None

    elif architecture == 2:
        inter_layer = False

        n_layers = 3
        d_rate = 0.1

        if inter_layer:
            c_temp = 2
        else:
            c_temp = 1

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+c_temp)
        activation_functions = [np.nan] * (n_layers+c_temp)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            hidden_layer_s[mm] = 256
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        if inter_layer:
            dropout_rates_tr[len(dropout_rates_tr)-1] = d_rate
            activation_functions[len(activation_functions)-2] = 'relu'
            activation_functions[len(activation_functions)-1] = 'relu'
        else:
            activation_functions[len(activation_functions)-1] = None
        
    elif architecture == 3:
        
        inter_layer = False

        n_layers = 1
        d_rate = 0.0

        if inter_layer:
            c_temp = 2
        else:
            c_temp = 1

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+c_temp)
        activation_functions = [np.nan] * (n_layers+c_temp)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            hidden_layer_s[mm] = 512
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        if inter_layer:
            dropout_rates_tr[len(dropout_rates_tr)-1] = d_rate
            activation_functions[len(activation_functions)-2] = 'relu'
            activation_functions[len(activation_functions)-1] = 'relu'
        else:
            activation_functions[len(activation_functions)-1] = None
    
    elif architecture == 4:

        inter_layer = False

        n_layers = 3
        d_rate = 0.0

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 256
            elif mm == 1:
                hidden_layer_s[mm] = 128
            elif mm == 2:
                hidden_layer_s[mm] = 64
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None
    
    elif architecture == 5:

        inter_layer = False

        n_layers = 3
        d_rate = 0.2

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 256
            elif mm == 1:
                hidden_layer_s[mm] = 128
            elif mm == 2:
                hidden_layer_s[mm] = 64
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None


    elif architecture == 6:

        inter_layer = False

        n_layers = 3
        d_rate = 0.0

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 32
            elif mm == 1:
                hidden_layer_s[mm] = 16
            elif mm == 2:
                hidden_layer_s[mm] = 8
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None

    elif architecture == 7:

        inter_layer = False

        n_layers = 3
        d_rate = 0.2

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 32
            elif mm == 1:
                hidden_layer_s[mm] = 16
            elif mm == 2:
                hidden_layer_s[mm] = 8
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None

    elif architecture == 8:

        inter_layer = False

        n_layers = 3
        d_rate = 0.2

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 32
            elif mm == 1:
                hidden_layer_s[mm] = 16
            elif mm == 2:
                hidden_layer_s[mm] = 8
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None

    elif architecture == 9:

        inter_layer = False

        n_layers = 3
        d_rate = 0.2

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 32
            elif mm == 1:
                hidden_layer_s[mm] = 16
            elif mm == 2:
                hidden_layer_s[mm] = 8
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None

    elif architecture == 10:

        inter_layer = False

        n_layers = 3
        d_rate = 0.2

        hidden_layer_s = [np.nan] * n_layers
        dropout_rates_tr = [np.nan] * (n_layers+1)
        activation_functions = [np.nan] * (n_layers+1)
        dropout_rates_tr[0] = d_rate
        for mm in range(0, n_layers):
            if mm == 0:
                hidden_layer_s[mm] = 256
            elif mm == 1:
                hidden_layer_s[mm] = 128
            elif mm == 2:
                hidden_layer_s[mm] = 64
            dropout_rates_tr[mm+1] = d_rate
            activation_functions[mm] = 'relu'
        
        activation_functions[len(activation_functions)-1] = None


    # Optimization options
    optimizer = 'Adam'
    max_iter = 2000                 # maximum number of iterations
    max_iter_nc = 50                # maximum number of iterations early stopping
    
    split_ratio = 0.3               # split ratio for training and validation set
    batch_s = 256                   # batch size
    use_bias = False                # include bias in neural net, if true

    # Define early stopping criterion
    early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=max_iter_nc, mode='min')

    learning_rate = opt['learning_rate']
    reg_par_w = opt['reg_par_w']          # regularization parameter for weights
    reg_par_b = opt['reg_par_b']          # regularization parameter for bias
    el_r_pro = opt['el_r_pro']            # split between elastic net and lasso: 1 - only l1 norm

    """
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    learning_rate,
    decay_steps=1,
    decay_rate=1,
    staircase=False)
    """

    # Create sparse neural network and compile it
    neural_net = sparse_nn(hidden_layer_s, activation_functions, dropout_rates_tr, num_s, num_f,
                reg_par_w, reg_par_b, el_r_pro, max_iter_nc, max_iter, optimizer, learning_rate, use_bias, inter_layer)

    neural_net.build_neural_network()
    # Compile and fit model
    neural_net.compile_nn()

    fit_nn = neural_net.model.fit(data_factor, data, epochs=max_iter,
                        validation_split=split_ratio, shuffle=True, batch_size=batch_s,
                        callbacks=early_stopping, verbose=0)
    
    # Compute market sensitivity
    with tf.GradientTape() as tape:
        data_factor_ts = tf.convert_to_tensor(data_factor[-1,:].reshape(1,-1), dtype=tf.float32)
        tape.watch(data_factor_ts)
        y_pred = neural_net.model(data_factor_ts)

    market_sens = tape.jacobian(y_pred, data_factor_ts)[-1,:,-1,:].numpy() 

    # Store for analysis
    market_return_t = data_factor_ts.numpy()[-1, :].reshape(1,-1)
    market_sensitivity = (market_sens, market_return_t)

    res = {'neural_net': neural_net, 'opt': opt, 'market_sensitivity': market_sensitivity}
    return res

#-------------------------------------------

def DNN_FM_cov(data, data_factor, DNN_model, c_err_cov, check_eig=False):

    num_n, num_s = data.shape

    y_hat_nn = DNN_model.model(data_factor).numpy()
    resd_nn = data - y_hat_nn

    sig_hat_e = thres_resd_new(resd_nn, c_err_cov, num_s, num_n)
    cov_f_nnet = np.cov(y_hat_nn.T)

    if not check_eig:
                
        sigma_y_nnet = cov_f_nnet + sig_hat_e
    else:

        cond = True
        const_c = 0
        while cond:

            sig_hat_e = thres_resd_new(resd_nn, c_err_cov+const_c, num_s, num_n)

            cond = (round(min(np.linalg.eig(sig_hat_e)[0]),2) < 0.01) or (np.linalg.cond(sig_hat_e) > num_s*10)
            const_c+=0.01
        
        sigma_y_nnet = cov_f_nnet + sig_hat_e

    inv_sigma_y_nnet = sig_inv_f_nnet(cov_f_nnet, sig_hat_e)

    res = {'sigma_hat': sigma_y_nnet, 'sigma_f_hat': cov_f_nnet, 'sigma_e_hat': sig_hat_e, 
           'inv_sigma_hat': inv_sigma_y_nnet}

    return res

def DNN_FM_cov_e_cmin(data, data_factor, DNN_model):

    num_n, num_s = data.shape

    y_hat_nn = DNN_model.model(data_factor).numpy()
    resd_nn = data - y_hat_nn

    num_n, num_s = resd_nn.shape

    f = lambda c: mineig_cov_e(resd=resd_nn, c_err_cov=c, num_s=num_s, num_n=num_n)

    if (f(50) * f(-50) < 0):
        r = brentq(f, -50, 50)
        return max(0, r)
    else:
        c = 0
        return c

def mineig_cov_e(resd, c_err_cov, num_s, num_n):

    return min(np.linalg.eig(thres_resd_new(resd, c_err_cov, num_s, num_n))[0])


#-------------------------------------------

# Function for determining optimal regularization and learning rate based on cross-validation or fixed
def opt_hyper_parameters(data, data_F, architecture, const_err_cov, use_CV_err, eval_type):
    
    parallel = False

    if architecture == 1 or architecture == 5 or architecture == 7:
        reg_par_w = 0.0005           # regularization parameter for weights
        reg_par_b = 0.0005           # regularization parameter for bias
    elif architecture == 9 or architecture == 10:
        reg_par_w = 0.005           # regularization parameter for weights
        reg_par_b = 0.005           # regularization parameter for bias
    else:
        reg_par_w = 0.0             # regularization parameter for weights
        reg_par_b = 0.0             # regularization parameter for bias
    el_r_pro = 1                    # split between elastic net and lasso: 1 - only l1 norm
    # Alternative specification for the learning rate
    learning_rate = 0.0005

    if use_CV_err:

        range_cov_err = np.arange(0, const_err_cov+0.6, 0.1)

        """
        opt = block_cv(data=data, data_F=data_F, architecture=architecture, 
                       reg_par=reg_par_w, lr=learning_rate, range_cov_err=range_cov_err, 
                       eval_type=eval_type, test_size=10, parallel=parallel)
        """
        
        opt = cv_split(data=data, data_F=data_F, architecture=architecture, 
                       reg_par=reg_par_w, lr=learning_rate, range_cov_err=range_cov_err, 
                       eval_type=eval_type)

    else:

        opt = {'learning_rate': learning_rate, 'reg_par_w': reg_par_w, 
            'reg_par_b': reg_par_b, 'el_r_pro': el_r_pro, 'const_err_cov': const_err_cov}
    
    return opt
        
#-------------------------------------------

# Function that determines hyperparameters based on block cross-validation

def block_cv(data, data_F, architecture, reg_par, lr, range_cov_err, eval_type, min_train_ratio=0.8, test_size=5, parallel=False):

    start = time.time()

    T, p = data.shape
    train_size = int(np.floor(min_train_ratio * T)) # Get training size
    numBlocks = int(np.floor((T - train_size) / test_size)) # Compute number of blocks
    
    # Show error message if test size is too large
    assert numBlocks > 1, f"Test size is too large"

    # Predefine dictionary for options
    opt = {'learning_rate': lr, 'reg_par_w': reg_par, 'reg_par_b': reg_par, 'el_r_pro': 1}
    
    res_mat = np.empty((numBlocks,len(range_cov_err)))
    res_mat[:] = np.nan

    for bl in range(1, numBlocks+1):
        idxTrainX = list(range((bl-1)*test_size,train_size+(bl-1)*test_size))

        if bl == numBlocks:
            idxTestX = list(range(train_size+(bl-1)*test_size,T))
        else:
            idxTestX = list(range(train_size+(bl-1)*test_size,train_size+bl*test_size))

        
        data_ntrain, data_mean, data_std = normalize_dat(data[idxTrainX,:])
        data_F_ntrain, data_F_mean, data_F_std = normalize_dat(data_F[idxTrainX,:])    

        test_res = np.empty((len(idxTestX),len(range_cov_err)))
        test_res[:] = np.nan
        
        # if bl == 1:
        res_DNN_FM = DNN_FM_core(data_ntrain, data_F_ntrain, architecture, opt)
        neural_net = res_DNN_FM['neural_net']


        for idx_t in range(1, len(idxTestX)+1):

            ind_test_temp = idxTrainX[idx_t:]+idxTestX[:idx_t]

            data_ntest, data_mean, data_std = normalize_dat(data[ind_test_temp,:])
            data_F_ntest, data_F_mean, data_F_std = normalize_dat(data_F[ind_test_temp,:])

            y_hat = neural_net.model(data_F_ntrain).numpy()
            resd_nn = data_ntrain - y_hat

            num_n, num_s = resd_nn.shape

            sigma_test = cov_sfm(data_ntest - neural_net.model(data_F_ntest).numpy())

            """
            # define the number of worker processes to use
            num_processes = 2

            combined_list = [(resd_nn, sigma_test, eval_type, const_err_item) for const_err_item in range_cov_err]

            if parallel:
                # create a pool of worker processes
                with get_context("spawn").Pool(processes=num_processes) as pool:
                    ret = pool.imap(err_cv_core, combined_list)
                    pool.close()
                    pool.join()
            else:
                ret = map(err_cv_core, combined_list)

            for indx, err_measure in enumerate(ret):
                test_res[idx_t-1,indx] = err_measure
            """

            # sigma_test = cov_sfm(data_ntest - neural_net.model(data_F_ntest).numpy())

            rate_thres = np.sqrt((np.log(num_s))/num_n)
            sig_e_samp = np.cov(resd_nn.T)
            thet_par = np.empty((num_s, num_s))
            thet_par[:] = np.nan

            for ii in range(0, num_s):
                for jj in range(0, num_s):
                    thet_par[ii, jj] = np.mean(np.abs(resd_nn[:, ii] * resd_nn[:, jj] - sig_e_samp[ii, jj]))

            sig_e_diag = np.diag(sig_e_samp)
            
            """
            sig_e_diag = np.diag(np.diag(sig_e_samp)**(0.5))
            R = np.linalg.inv(sig_e_diag) @ sig_e_samp @ np.linalg.inv(sig_e_diag)
            """
            
            for c_idx in range(0,len(range_cov_err)):
                lam = rate_thres * range_cov_err[c_idx] * thet_par
                
                """
                M = soft_t(R, lam)
                M = M - np.diag(np.diag(M)) + np.eye(num_s)
                sig_hat_e = sig_e_diag @ M @ sig_e_diag
                """
                
                sig_hat_e = soft_t(sig_e_samp, lam)
                np.fill_diagonal(sig_hat_e, sig_e_diag)
                
                # sig_hat_e = thres_resd_new(resd_nn, range_cov_err[c_idx], num_s, num_n)

                if min(np.linalg.eig(sig_hat_e)[0]) < 0:
                    test_res[idx_t-1,c_idx] = np.inf
                else:
                    if eval_type == 'frob':
                        test_res[idx_t-1,c_idx] = np.linalg.norm(sig_hat_e - sigma_test, ord='fro')**2
                    elif eval_type == 'spec':
                        test_res[idx_t-1,c_idx] = np.linalg.norm(sig_hat_e - sigma_test, ord=2)**2

            
        res_mat[bl-1,:] = np.mean(test_res, axis=0)


     
    idx_opt = np.where(res_mat.mean(axis=0) == np.nanmin(res_mat.mean(axis=0)))

    opt.update({'const_err_cov': range_cov_err[idx_opt][0]})
    end = time.time()

    print(end - start)
    return opt

def err_cv_core(tuple_in):

    resd_nn, sigma_test, eval_type, const_err_cov = tuple_in[0], tuple_in[1], tuple_in[2], tuple_in[3]

    num_n, num_s = resd_nn.shape

    st = time.time()
    sig_hat_e = thres_resd_new(resd_nn, const_err_cov, num_s, num_n)
    print(time.time()-st)

    if min(np.linalg.eig(sig_hat_e)[0]) < 0:
        test_res = np.inf
    else:
        if eval_type == 'frob':
            test_res = np.linalg.norm(sig_hat_e - sigma_test, ord='fro')
        elif eval_type == 'spec':
            st1 = time.time()
            test_res = np.linalg.norm(sig_hat_e - sigma_test, ord=2)
            print(time.time()-st1)

    return test_res

def cv_split(data, data_F, architecture, reg_par, lr, range_cov_err, eval_type):

    start = time.time()

    # Predefine dictionary for options
    opt = {'learning_rate': lr, 'reg_par_w': reg_par, 'reg_par_b': reg_par, 'el_r_pro': 1}

    data_dm, _, _ = normalize_dat_sim(data)
    data_F_dm, _, _ = normalize_dat_sim(data_F)    

    res_DNN_FM = DNN_FM_core(data_dm, data_F_dm, architecture, opt)
    neural_net = res_DNN_FM['neural_net']

    y_hat = neural_net.model(data_F_dm).numpy()
    resd_nn = data_dm - y_hat

    n_folds = 10

    res_mat = np.empty((n_folds,len(range_cov_err)))
    res_mat[:] = np.nan

    split_sample, _ = ts_train_test_split(resd_nn, n_folds, train_size=0.5)

    for m_idx in range(0, n_folds):


            resd_nn_s1 = split_sample[m_idx][0]
            resd_nn_s2 = split_sample[m_idx][1]

            num_n, num_s = resd_nn_s1.shape

            sigma_test = cov_sfm(resd_nn_s2) # np.cov(resd_nn_s2.T) # 

            sig_e_samp, thet_par = thres_cov_resd_aux(resd_nn_s1, num_s)

            for c_idx in range(0,len(range_cov_err)):
                
                sig_hat_e = thres_cov_resd(sig_e_samp, thet_par, range_cov_err[c_idx], num_s, num_n)

                if min(np.linalg.eig(sig_hat_e)[0]) < 0:
                    res_mat[m_idx,c_idx] = np.inf
                else:
                    if eval_type == 'frob':
                        res_mat[m_idx,c_idx] = np.linalg.norm(sig_hat_e - sigma_test, ord='fro')**2
                    elif eval_type == 'spec':
                        res_mat[m_idx,c_idx] = np.linalg.norm(sig_hat_e - sigma_test, ord=2)**2

    idx_opt = np.where(res_mat.mean(axis=0) == np.nanmin(res_mat.mean(axis=0)))

    opt.update({'const_err_cov': range_cov_err[idx_opt][0]})
    end = time.time()

    print(end - start)

    return opt

def ts_train_test_split(X, n_folds = 5, train_size=0.5):

    test_size = 1 - train_size
    n_obs = X.shape[0]

    size_split = n_obs - n_folds

    n_train = round(size_split * train_size)
    n_test = round(size_split * test_size)

    split_t = (list(range(0, n_train)), list(range(n_train, n_train+n_test)))

    split_sample = []
    split_index = []

    for jj in range(0, n_folds):
        split_index.append(([el1 + jj for el1 in split_t[0]], [el2 + jj for el2 in split_t[1]]))
        split_sample.append((X[split_index[jj][0],:], X[split_index[jj][1],:]))

    return split_sample, split_index


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star

def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    long_only : bool
        If True, falls back to GMV if MV produces negative weights
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        print('SINGULARITY')
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star

def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        print('WARNING: Weight sum near zero, returning equal weights')
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star


def load_yearly_signals(year, buys_path_template='buys_{}.csv', sells_path_template='sells_{}.csv'):
    """
    Load buy and sell signals for a specific year.
    
    Parameters:
    -----------
    year : int
        Year to load signals for
    buys_path_template : str
        Template for buys file path (use {} for year placeholder)
    sells_path_template : str
        Template for sells file path (use {} for year placeholder)
    
    Returns:
    --------
    permno_set : set
        Set of permnos in the buy and sell signals for this year
    """
    try:
        buys = pd.read_csv(buys_path_template.format(year), index_col=1)
        sells = pd.read_csv(sells_path_template.format(year), index_col=1)
        
        buys.index.name = 'permno'
        sells.index.name = 'permno'
        
        buys_index = buys.index.astype(int)
        sells_index = sells.index.astype(int)
        
        return set(buys_index.union(sells_index))
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load signals for year {year}: {e}")
        return set()

def load_finbert_signals(signals_path):
    """
    Load FinBERT monthly signals from CSV file.
    
    Parameters:
    -----------
    signals_path : str
        Path to monthly_signals.csv file
    
    Returns:
    --------
    signals_df : pd.DataFrame
        DataFrame with columns: symbol, company, year_month, signal, avg_sentiment_score
    """
    try:
        signals_df = pd.read_csv(signals_path)
        # Convert year_month to datetime (end of month)
        signals_df['date'] = pd.to_datetime(signals_df['year_month']) + pd.offsets.MonthEnd(0)
        return signals_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load FinBERT signals: {e}")
        return pd.DataFrame(columns=['symbol', 'company', 'year_month', 'signal', 'date'])

def get_finbert_permnos_for_date(signals_df, ticker_to_permno, date):
    """
    Get set of permnos with 'buy' or 'sell' signals for a specific date.
    
    Parameters:
    -----------
    signals_df : pd.DataFrame
        FinBERT signals dataframe
    ticker_to_permno : dict
        Mapping from ticker symbol to permno
    date : pd.Timestamp
        Date to get signals for
    
    Returns:
    --------
    permno_set : set
        Set of permnos with buy or sell signals on this date
    """
    # Get signals for this date
    date_signals = signals_df[signals_df['date'] == date]
    
    # Filter for buy and sell signals (exclude hold)
    buy_signals = date_signals[date_signals['signal'] == 'buy']
    sell_signals = date_signals[date_signals['signal'] == 'sell']
    
    # Convert tickers to permnos
    permnos = set()
    for ticker in buy_signals['symbol'].values:
        if ticker in ticker_to_permno:
            permnos.add(ticker_to_permno[ticker])
    for ticker in sell_signals['symbol'].values:
        if ticker in ticker_to_permno:
            permnos.add(ticker_to_permno[ticker])
    
    return permnos


def create_ticker_to_permno_mapping(df):
    """
    Create a mapping from ticker to permno from the returns dataframe.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Returns dataframe with 'ticker' and 'permno' columns
    
    Returns:
    --------
    ticker_to_permno : dict
        Mapping from ticker to permno (uses most recent permno for each ticker)
    """
    if 'ticker' not in df.columns:
        raise ValueError("DataFrame must have 'ticker' column for mapping")
    
    # Drop NaN tickers
    valid_df = df[df['ticker'].notna()].copy()
    
    # Get the most recent permno for each ticker
    ticker_to_permno = valid_df.groupby('ticker')['permno'].last().to_dict()
    
    return ticker_to_permno

def load_analyst_recommendations(rec_changes_path):
    """
    Load significant recommendation changes from CSV file.
    
    Parameters:
    -----------
    rec_changes_path : str
        Path to significant_recommendation_changes.csv file
    
    Returns:
    --------
    rec_changes_df : pd.DataFrame
        DataFrame with columns: permno, date, ticker, mean_recommendation, 
        recommendation_change, num_recommendations
    """
    try:
        rec_changes_df = pd.read_csv(rec_changes_path)
        rec_changes_df['date'] = pd.to_datetime(rec_changes_df['date'])
        rec_changes_df['permno'] = rec_changes_df['permno'].astype(int)
        return rec_changes_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load recommendation changes: {e}")
        return pd.DataFrame(columns=['permno', 'date', 'ticker', 'mean_recommendation', 
                                    'recommendation_change', 'num_recommendations'])


def get_signal_permnos_for_date(rec_changes_df, date, buy_threshold=-0.5, sell_threshold=0.5):
    """
    Get sets of permnos with buy/sell signals based on recommendation changes.
    
    Note: Negative change = upgrade (moving toward Strong Buy) = BUY signal
          Positive change = downgrade (moving toward Sell) = SELL signal
    
    Parameters:
    -----------
    rec_changes_df : pd.DataFrame
        Recommendation changes dataframe
    date : pd.Timestamp
        Date to get signals for
    buy_threshold : float
        Threshold for buy signals (default: -0.5)
    sell_threshold : float
        Threshold for sell signals (default: 0.5)
    
    Returns:
    --------
    buy_permnos : set
        Set of permnos with buy signals
    sell_permnos : set
        Set of permnos with sell signals
    """
    date_changes = rec_changes_df[rec_changes_df['date'] == date]
    
    # Buy signals: negative changes (recommendations getting better)
    buys = date_changes[date_changes['recommendation_change'] <= buy_threshold]
    buy_permnos = set(buys['permno'].values)
    
    # Sell signals: positive changes (recommendations getting worse)
    sells = date_changes[date_changes['recommendation_change'] >= sell_threshold]
    sell_permnos = set(sells['permno'].values)
    
    return buy_permnos | sell_permnos


def calculate_exit_transaction_cost(prev_weights_dict, prev_oos_returns_dict, 
                                    prev_gross_return, transaction_cost, verbose=False):
    """Calculate transaction cost when exiting the market (liquidating all positions)."""
    if len(prev_weights_dict) == 0:
        return 0.0, 0.0, 0.0
    
    adjusted_prev = {}
    for asset, prev_w in prev_weights_dict.items():
        if asset in prev_oos_returns_dict:
            prev_r = prev_oos_returns_dict[asset]
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
        else:
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
    
    turnover = sum(abs(w) for w in adjusted_prev.values())
    tc = transaction_cost * 1.0 * turnover
    net_return = -tc
    
    if verbose:
        print(f"  Liquidating positions | Turnover: {turnover:>6.4f} | TC: {tc:>8.6f}")
    
    return turnover, tc, net_return



def backtest_dnn_yearly(df, 
                          test_start_date='2020-01-31', 
                          test_end_date='2024-11-30',
                          lookback_window=180,
                          transaction_cost=0.001,
                          buys_path_template='buys_{}.csv',
                          sells_path_template='sells_{}.csv',
                          analyst_rec_path=None,
                          finbert_signals_path=None,
                          data_factor=None,
                          verbose=True):
    """
    Backtest DNN-FM with FinBERT + LLM signals using GMV/MV/MSR strategies.
    
    KEY CHANGES:
    1. Added finbert_signals_path parameter
    2. Added ticker to permno mapping creation
    3. Added FinBERT signal loading and processing
    4. Combined yearly and FinBERT signals using union/intersection logic
    5. Added calculate_exit_transaction_cost for proper liquidation handling
    """
    
    # [Keep all existing setup code]
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])

    # Create ticker to permno mapping
    if verbose:
        print("Creating ticker to permno mapping...")
    ticker_to_permno = create_ticker_to_permno_mapping(df)
    if verbose:
        print(f"Mapped {len(ticker_to_permno)} unique tickers to permnos")
    
    # Load FinBERT signals if provided
    finbert_df = None
    if finbert_signals_path is not None:
        finbert_df = load_finbert_signals(finbert_signals_path)
        if verbose and len(finbert_df) > 0:
            print(f"Loaded FinBERT signals: {len(finbert_df)} monthly records")
            print(f"FinBERT signal distribution:")
            print(finbert_df['signal'].value_counts())
    
    # Load analyst recommendations if provided
    analyst_df = None
    if analyst_rec_path is not None:
        analyst_df = load_analyst_recommendations(analyst_rec_path)
        if verbose and len(analyst_df) > 0:
            print(f"Loaded analyst recommendations: {len(analyst_df)} records")
    
    
    # [Keep all existing date and storage setup]
    all_dates = sorted(df['datadate'].unique())
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    # Storage for results - GMV
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # Storage for results - MV
    portfolio_returns_2 = []
    portfolio_dates_2 = []
    portfolio_weights_list_2 = []
    portfolio_turnover_list_2 = []
    portfolio_gross_returns_2 = []
    
    # Storage for results - MSR
    portfolio_returns_3 = []
    portfolio_dates_3 = []
    portfolio_weights_list_3 = []
    portfolio_turnover_list_3 = []
    portfolio_gross_returns_3 = []
    
    # Track weights by permno - GMV
    prev_weights_dict = {}
    prev_oos_returns_dict = {}
    prev_gross_return = 0.0
    
    # Track weights by permno - MV
    prev_weights_dict_2 = {}
    prev_oos_returns_dict_2 = {}
    prev_gross_return_2 = 0.0
    
    # Track weights by permno - MSR
    prev_weights_dict_3 = {}
    prev_oos_returns_dict_3 = {}
    prev_gross_return_3 = 0.0
    
    # Cache for yearly signals
    yearly_signals_cache = {}
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST WITH DNN-FM + YEARLY + ANALYST SIGNALS")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        current_year = current_date.year
        
        # Load yearly signals
        if current_year not in yearly_signals_cache:
            yearly_signals_cache[current_year] = load_yearly_signals(
                current_year, buys_path_template, sells_path_template
            )
        
        yearly_permnos = yearly_signals_cache[current_year]
        
        # Get analyst recommendations for current date
        analyst_permnos = set()
        if analyst_df is not None and len(analyst_df) > 0:
            analyst_permnos = get_signal_permnos_for_date(analyst_df, current_date)

        # Get FinBERT signals for current date
        finbert_permnos = set()
        if finbert_df is not None and len(finbert_df) > 0:
            finbert_permnos = get_finbert_permnos_for_date(finbert_df, ticker_to_permno, current_date)
        
        # keep only permnos that 2 strats agree on
        allowed_permnos = (yearly_permnos & analyst_permnos) | \
                          (yearly_permnos & finbert_permnos) | \
                          (analyst_permnos & finbert_permnos)
        
        # NEW: Get OOS returns FIRST (critical for exit transaction cost calculation)
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # NEW: Handle no signals case with proper liquidation
        if len(allowed_permnos) == 0:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ No signals, recording zero return for all strategies")
            
            # Liquidate all three strategies
            for idx, (pw, po, pg) in enumerate([
                (prev_weights_dict, prev_oos_returns_dict, prev_gross_return),
                (prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2),
                (prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3)
            ]):
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pw, po, pg, transaction_cost, verbose=verbose
                )
                
                if idx == 0:  # GMV
                    portfolio_returns.append(net_return)
                    portfolio_dates.append(current_date)
                    portfolio_weights_list.append({})
                    portfolio_turnover_list.append(turnover)
                    portfolio_gross_returns.append(0.0)
                elif idx == 1:  # MV
                    portfolio_returns_2.append(net_return)
                    portfolio_dates_2.append(current_date)
                    portfolio_weights_list_2.append({})
                    portfolio_turnover_list_2.append(turnover)
                    portfolio_gross_returns_2.append(0.0)
                else:  # MSR
                    portfolio_returns_3.append(net_return)
                    portfolio_dates_3.append(current_date)
                    portfolio_weights_list_3.append({})
                    portfolio_turnover_list_3.append(turnover)
                    portfolio_gross_returns_3.append(0.0)
            
            # Reset all state
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            continue
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        train_factor = data_factor.loc[window_start_date : window_end_date]
        
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')} | Year: {current_year}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            # NEW: Show both signal types
            print(f"  Yearly: {len(yearly_permnos)} | Analyst: {len(analyst_permnos)} | FinBERT: {len(finbert_permnos)} | "
                  f"Union/Intersection: {len(allowed_permnos)} | Assets w/ data: {p_current}")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), recording 0 return")
            # Liquidate all three strategies
            for idx, (pw, po, pg) in enumerate([
                (prev_weights_dict, prev_oos_returns_dict, prev_gross_return),
                (prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2),
                (prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3)
            ]):
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pw, po, pg, transaction_cost, verbose=verbose
                )
                
                if idx == 0:  # GMV
                    portfolio_returns.append(net_return)
                    portfolio_dates.append(current_date)
                    portfolio_weights_list.append({})
                    portfolio_turnover_list.append(turnover)
                    portfolio_gross_returns.append(0.0)
                elif idx == 1:  # MV
                    portfolio_returns_2.append(net_return)
                    portfolio_dates_2.append(current_date)
                    portfolio_weights_list_2.append({})
                    portfolio_turnover_list_2.append(turnover)
                    portfolio_gross_returns_2.append(0.0)
                else:  # MSR
                    portfolio_returns_3.append(net_return)
                    portfolio_dates_3.append(current_date)
                    portfolio_weights_list_3.append({})
                    portfolio_turnover_list_3.append(turnover)
                    portfolio_gross_returns_3.append(0.0)
            
            # Reset all state
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            continue
        else:
            try:
                # Demean the returns
                Y_bar = Y.mean(axis=0)
                Y_star = Y - Y_bar
                
                if verbose:
                    print(f"  Running Deep Learning Regression...")
                F = train_factor.values.astype(float)
                res_nnet_fm = DNN_FM_main(Y_star, F, architecture=5, const_err_cov=2.5, 
                                         use_CV_err=False, eval_type='frob')
                Theta_hat = res_nnet_fm['inv_sigma_hat']
                
                if verbose:
                    print(f"  Computing GMV weights...")
                w_star = gmv_weights(Theta_hat)
                
                if verbose:
                    print(f"  Computing MV weights...")
                w_star_2 = mv_weights(Theta_hat, Y_bar, target_return=0.01)
                
                if verbose:
                    print(f"  Computing MSR weights...")
                w_star_3 = msr_weights(Theta_hat, Y_bar)
                
                # Create weights dictionaries
                new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
                new_weights_dict_2 = {asset: w_star_2[i] for i, asset in enumerate(current_assets)}
                new_weights_dict_3 = {asset: w_star_3[i] for i, asset in enumerate(current_assets)}
                
            except Exception as e:
                if verbose:
                    print(f"  ✗ Error: {e}")
                    print(f"  Using previous weights")
                new_weights_dict = prev_weights_dict.copy()
                new_weights_dict_2 = prev_weights_dict_2.copy()
                new_weights_dict_3 = prev_weights_dict_3.copy()

        # Normalize weights to sum to 1 - GMV
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ GMV: Zero weight sum, using previous weights")
            new_weights_dict = prev_weights_dict.copy()
            weight_sum = sum(new_weights_dict.values())
            if weight_sum > 1e-10:
                new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        
        # Normalize weights to sum to 1 - MV
        weight_sum_2 = sum(new_weights_dict_2.values())
        if weight_sum_2 > 1e-10:
            new_weights_dict_2 = {k: v/weight_sum_2 for k, v in new_weights_dict_2.items()}
        else:
            if verbose:
                print("  ⚠ MV: Zero weight sum, using previous weights")
            new_weights_dict_2 = prev_weights_dict_2.copy()
            weight_sum_2 = sum(new_weights_dict_2.values())
            if weight_sum_2 > 1e-10:
                new_weights_dict_2 = {k: v/weight_sum_2 for k, v in new_weights_dict_2.items()}
        
        # Normalize weights to sum to 1 - MSR
        weight_sum_3 = sum(new_weights_dict_3.values())
        if weight_sum_3 > 1e-10:
            new_weights_dict_3 = {k: v/weight_sum_3 for k, v in new_weights_dict_3.items()}
        else:
            if verbose:
                print("  ⚠ MSR: Zero weight sum, using previous weights")
            new_weights_dict_3 = prev_weights_dict_3.copy()
            weight_sum_3 = sum(new_weights_dict_3.values())
            if weight_sum_3 > 1e-10:
                new_weights_dict_3 = {k: v/weight_sum_3 for k, v in new_weights_dict_3.items()}
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Get out-of-sample returns for current month (only for allowed permnos)
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        
        # Filter out NaN returns
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Find common assets between weights and returns
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        common_assets_2 = set(new_weights_dict_2.keys()) & set(oos_returns_dict.keys())
        common_assets_3 = set(new_weights_dict_3.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0 or len(common_assets_2) == 0 or len(common_assets_3) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, skipping period")
            continue
        
        # Filter to common assets and renormalize - GMV
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ GMV: Zero weight sum after filtering, skipping period")
            continue
        
        # Filter to common assets and renormalize - MV
        common_weights_2 = {a: new_weights_dict_2[a] for a in common_assets_2}
        common_weight_sum_2 = sum(common_weights_2.values())
        if common_weight_sum_2 > 1e-10:
            common_weights_2 = {k: v/common_weight_sum_2 for k, v in common_weights_2.items()}
        else:
            if verbose:
                print("  ⚠ MV: Zero weight sum after filtering, skipping period")
            continue
        
        # Filter to common assets and renormalize - MSR
        common_weights_3 = {a: new_weights_dict_3[a] for a in common_assets_3}
        common_weight_sum_3 = sum(common_weights_3.values())
        if common_weight_sum_3 > 1e-10:
            common_weights_3 = {k: v/common_weight_sum_3 for k, v in common_weights_3.items()}
        else:
            if verbose:
                print("  ⚠ MSR: Zero weight sum after filtering, skipping period")
            continue
        
        # Compute gross portfolio returns
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        gross_return_2 = sum(common_weights_2[a] * oos_returns_dict[a] for a in common_assets_2)
        gross_return_3 = sum(common_weights_3[a] * oos_returns_dict[a] for a in common_assets_3)
        
        # Sanity checks
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ GMV: Invalid gross return: {gross_return}, skipping period")
            continue
        if np.isnan(gross_return_2) or np.isinf(gross_return_2):
            if verbose:
                print(f"  ⚠ MV: Invalid gross return: {gross_return_2}, skipping period")
            continue
        if np.isnan(gross_return_3) or np.isinf(gross_return_3):
            if verbose:
                print(f"  ⚠ MSR: Invalid gross return: {gross_return_3}, skipping period")
            continue
        
        # === IMPROVED TRANSACTION COST CALCULATION - GMV ===
        if len(prev_weights_dict) > 0:
            # Step 1: Adjust ALL previous weights for their returns
            adjusted_prev = {}
            
            for asset, prev_w in prev_weights_dict.items():
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    # Asset had weight but no return data (exited)
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
            
            # Step 2: Calculate turnover across all assets (old and new)
            all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
            
            turnover = 0.0
            for asset in all_assets:
                old_w = adjusted_prev.get(asset, 0.0)
                new_w = common_weights.get(asset, 0.0)
                turnover += abs(new_w - old_w)
            
            # Transaction cost on end-of-period portfolio value
            tc = transaction_cost * (1 + gross_return) * turnover
        else:
            # First period: buying into everything
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * (1 + gross_return) * turnover
        
        # === IMPROVED TRANSACTION COST CALCULATION - MV ===
        if len(prev_weights_dict_2) > 0:
            adjusted_prev_2 = {}
            
            for asset, prev_w in prev_weights_dict_2.items():
                if asset in prev_oos_returns_dict_2:
                    prev_r = prev_oos_returns_dict_2[asset]
                    if abs(1 + prev_gross_return_2) > 1e-6:
                        adjusted_prev_2[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return_2)
                    else:
                        adjusted_prev_2[asset] = 0.0
                else:
                    if abs(1 + prev_gross_return_2) > 1e-6:
                        adjusted_prev_2[asset] = prev_w / (1 + prev_gross_return_2)
                    else:
                        adjusted_prev_2[asset] = 0.0
            
            all_assets_2 = set(adjusted_prev_2.keys()) | set(common_weights_2.keys())
            
            turnover_2 = 0.0
            for asset in all_assets_2:
                old_w = adjusted_prev_2.get(asset, 0.0)
                new_w = common_weights_2.get(asset, 0.0)
                turnover_2 += abs(new_w - old_w)
            
            tc_2 = transaction_cost * (1 + gross_return_2) * turnover_2
        else:
            turnover_2 = sum(abs(w) for w in common_weights_2.values())
            tc_2 = transaction_cost * (1 + gross_return_2) * turnover_2
        
        # === IMPROVED TRANSACTION COST CALCULATION - MSR ===
        if len(prev_weights_dict_3) > 0:
            adjusted_prev_3 = {}
            
            for asset, prev_w in prev_weights_dict_3.items():
                if asset in prev_oos_returns_dict_3:
                    prev_r = prev_oos_returns_dict_3[asset]
                    if abs(1 + prev_gross_return_3) > 1e-6:
                        adjusted_prev_3[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return_3)
                    else:
                        adjusted_prev_3[asset] = 0.0
                else:
                    if abs(1 + prev_gross_return_3) > 1e-6:
                        adjusted_prev_3[asset] = prev_w / (1 + prev_gross_return_3)
                    else:
                        adjusted_prev_3[asset] = 0.0
            
            all_assets_3 = set(adjusted_prev_3.keys()) | set(common_weights_3.keys())
            
            turnover_3 = 0.0
            for asset in all_assets_3:
                old_w = adjusted_prev_3.get(asset, 0.0)
                new_w = common_weights_3.get(asset, 0.0)
                turnover_3 += abs(new_w - old_w)
            
            tc_3 = transaction_cost * (1 + gross_return_3) * turnover_3
        else:
            turnover_3 = sum(abs(w) for w in common_weights_3.values())
            tc_3 = transaction_cost * (1 + gross_return_3) * turnover_3
        
        # Net returns
        net_return = gross_return - tc
        net_return_2 = gross_return_2 - tc_2
        net_return_3 = gross_return_3 - tc_3
        
        # Store results - GMV
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Store results - MV
        portfolio_returns_2.append(net_return_2)
        portfolio_dates_2.append(current_date)
        portfolio_weights_list_2.append(common_weights_2.copy())
        portfolio_turnover_list_2.append(turnover_2)
        portfolio_gross_returns_2.append(gross_return_2)
        
        # Store results - MSR
        portfolio_returns_3.append(net_return_3)
        portfolio_dates_3.append(current_date)
        portfolio_weights_list_3.append(common_weights_3.copy())
        portfolio_turnover_list_3.append(turnover_3)
        portfolio_gross_returns_3.append(gross_return_3)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        prev_weights_dict_2 = common_weights_2.copy()
        prev_oos_returns_dict_2 = {a: oos_returns_dict[a] for a in common_assets_2}
        prev_gross_return_2 = gross_return_2
        
        prev_weights_dict_3 = common_weights_3.copy()
        prev_oos_returns_dict_3 = {a: oos_returns_dict[a] for a in common_assets_3}
        prev_gross_return_3 = gross_return_3
        
        if verbose:
            print(f"  GMV  - Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")
            print(f"  MV   - Gross: {gross_return_2:>8.5f} | Turnover: {turnover_2:>6.4f} | "
                  f"TC: {tc_2:>8.6f} | Net: {net_return_2:>8.5f}")
            print(f"  MSR  - Gross: {gross_return_3:>8.5f} | Turnover: {turnover_3:>6.4f} | "
                  f"TC: {tc_3:>8.6f} | Net: {net_return_3:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    results_df_2 = pd.DataFrame({
        'date': portfolio_dates_2,
        'portfolio_return': portfolio_returns_2,
        'portfolio_gross_return': portfolio_gross_returns_2,
        'portfolio_weights': portfolio_weights_list_2,
        'portfolio_turnover': portfolio_turnover_list_2
    })
    results_df_2['cumulative_return'] = (1 + results_df_2['portfolio_return']).cumprod() - 1
    
    results_df_3 = pd.DataFrame({
        'date': portfolio_dates_3,
        'portfolio_return': portfolio_returns_3,
        'portfolio_gross_return': portfolio_gross_returns_3,
        'portfolio_weights': portfolio_weights_list_3,
        'portfolio_turnover': portfolio_turnover_list_3
    })
    results_df_3['cumulative_return'] = (1 + results_df_3['portfolio_return']).cumprod() - 1
    
    # Helper function to compute metrics
    def compute_metrics(returns_list, turnover_list, results_df):
        if len(returns_list) > 0:
            mean_return = np.mean(returns_list)
            variance = np.var(returns_list, ddof=1)
            sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
            
            # Annualized metrics (monthly data)
            annual_return = mean_return * 12
            annual_volatility = np.sqrt(variance * 12)
            annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
            
            return {
                'mean_return': mean_return,
                'variance': variance,
                'sharpe_ratio': sharpe_ratio,
                'annual_return': annual_return,
                'annual_volatility': annual_volatility,
                'annual_sharpe_ratio': annual_sharpe,
                'total_return': results_df['cumulative_return'].iloc[-1],
                'avg_turnover': np.mean(turnover_list),
                'n_periods': len(returns_list)
            }
        else:
            return {
                'mean_return': 0, 'variance': 0, 'sharpe_ratio': 0,
                'annual_return': 0, 'annual_volatility': 0, 'annual_sharpe_ratio': 0,
                'total_return': 0, 'avg_turnover': 0, 'n_periods': 0
            }
    
    # Compute metrics for all three strategies
    metrics = compute_metrics(portfolio_returns, portfolio_turnover_list, results_df)
    metrics_2 = compute_metrics(portfolio_returns_2, portfolio_turnover_list_2, results_df_2)
    metrics_3 = compute_metrics(portfolio_returns_3, portfolio_turnover_list_3, results_df_3)
    
    return results_df, metrics, results_df_2, metrics_2, results_df_3, metrics_3

2025-12-27 14:01:10.584035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('../../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

data_f=pd.read_csv('F-F_Research_Data_Factors.csv',sep=',')
data_f['Date']=pd.to_datetime(data_f['Date'], format="%Y%m")
data_f['Date']=data_f['Date']+pd.offsets.MonthEnd(0)
data_f = data_f.set_index('Date')
data_f = data_f[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# Run backtest with yearly signals
results_df, metrics, results_df_2, metrics_2, results_df_3, metrics_3= backtest_dnn_yearly(
    df,
    test_start_date='2015-01-31',
    test_end_date='2024-04-30',
    lookback_window=180,
    transaction_cost=0.001,
    buys_path_template='buys_{}.csv',
    sells_path_template='sells_{}.csv',
    analyst_rec_path='../examples/monthly_mean_recommendations_decay.csv',
    finbert_signals_path='../examples/monthly_signals_decay.csv',  # Your FinBERT signals
    data_factor=data_f,
    verbose=True
)

print(f"\n GMV")
print(f"\nSharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics['total_return']:.4f}")
print(f"Average Turnover: {metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"\nSharpe Ratio: {metrics_2['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics_2['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics_2['total_return']:.4f}")
print(f"Average Turnover: {metrics_2['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"\nSharpe Ratio: {metrics_3['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics_3['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics_3['total_return']:.4f}")
print(f"Average Turnover: {metrics_3['avg_turnover']:.4f}")

Creating ticker to permno mapping...
Mapped 1664 unique tickers to permnos
Loaded FinBERT signals: 54240 monthly records
FinBERT signal distribution:
signal
hold    52823
sell      859
buy       558
Name: count, dtype: int64
Loaded analyst recommendations: 37527 records
STARTING BACKTEST WITH DNN-FM + YEARLY + ANALYST SIGNALS

[1/112] Date: 2015-01-31 | Year: 2015
  Window: 2000-01-31 to 2014-12-31
  Yearly: 54 | Analyst: 69 | FinBERT: 10 | Union/Intersection: 10 | Assets w/ data: 7
  Running Deep Learning Regression...
2.5
  Computing GMV weights...
  Computing MV weights...
  Computing MSR weights...
  GMV  - Gross:  0.04100 | Turnover: 1.0000 | TC: 0.001041 | Net:  0.03996
  MV   - Gross:  0.05005 | Turnover: 1.0428 | TC: 0.001095 | Net:  0.04895
  MSR  - Gross:  0.04839 | Turnover: 1.0208 | TC: 0.001070 | Net:  0.04732

[2/112] Date: 2015-02-28 | Year: 2015
  Window: 2000-02-29 to 2015-01-31
  Yearly: 54 | Analyst: 91 | FinBERT: 4 | Union/Intersection: 7 | Assets w/ data: 4
  Runni

In [3]:
print(f"\n GMV")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics['mean_return']*12:.4f}")
print(f"Variance: {metrics['variance']*12:.4f}")
print(f"Avg Turnover: {metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"Annualized Sharpe Ratio: {metrics_2['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics_2['mean_return']*12:.4f}")
print(f"Variance: {metrics_2['variance']*12:.4f}")
print(f"Avg Turnover: {metrics_2['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"Annualized Sharpe Ratio: {metrics_3['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics_3['mean_return']*12:.4f}")
print(f"Variance: {metrics_3['variance']*12:.4f}")
print(f"Avg Turnover: {metrics_3['avg_turnover']:.4f}")


 GMV
Annualized Sharpe Ratio: 0.4200
Mean Return: 0.0671
Variance: 0.0255
Avg Turnover: 1.6766

 MV
Annualized Sharpe Ratio: 0.0983
Mean Return: 0.0174
Variance: 0.0315
Avg Turnover: 2.0509

 MSR
Annualized Sharpe Ratio: 0.1869
Mean Return: 0.0317
Variance: 0.0287
Avg Turnover: 1.8557
